In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, cross_val_score, train_test_split
from sklearn import set_config
set_config(display="diagram")

In [8]:
data = sns.load_dataset("tips")
#data.info()
#data.describe()
data2 = data.drop("tip", axis = 1)
X = data.drop("tip", axis = 1)
y = data["tip"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

 - Make_column_selector = Permet de récupérer les colonnes en fonctions du types
 - Make_column_transformer = Permet de scale les colonnes
 - Pipeline = Elément du processeur où les instructions sont découpées en plusieurs tâches afin d'être exécutées

In [9]:
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector


num_selector = make_column_selector(dtype_include=["float", "int64"])
cat_selector = make_column_selector(dtype_include="category")
cat_selector(data2)
num_selector(data2)

['total_bill', 'size']

In [10]:
cat_processor = OneHotEncoder(handle_unknown="ignore")
num_processor = Pipeline([
    ("imputer", SimpleImputer(strategy= "mean")),
    ("MinMaxScaler", MinMaxScaler())
])

In [11]:
preprocessor = make_column_transformer(
    (num_processor, num_selector), (cat_processor, cat_selector)
)
preprocessor

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x283fc1190>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x283fc1e80>)])

In [12]:
model = make_pipeline(preprocessor, LinearRegression())
model.fit(X_train, y_train)
model.score(X_test, y_test)
#model.predict(X_test)

0.4465333278367788

In [14]:
cross_validate(model, X_train, y_train, cv=5)["test_score"].mean()
cross_val_score(model, X_train, y_train).mean()


0.32072287345350836

In [15]:
model.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('MinMaxScaler',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x283fc1190>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x283fc1e80>)])),
                ('linearregression', LinearRegression())])

In [16]:
import pickle 

pickle.dump(model, open("pipeline_deploy.pkl", "wb"))
pickled_model = pickle.load(open("pipeline_deploy.pkl", "rb"))
#pickled_model.score(X,y) Inutile à faire 

In [17]:
def get_user_df(total_bill, sex, smoker, day, time, size):
    x_user = pd.DataFrame({
        "total_bill" : [float(total_bill)],
        "sex" : [sex.capitalize()],
        "smoker" : [smoker.capitalize()],
        "day" : [day.capitalize()],
        "time" : [time.capitalize()],
        "size" : [int(size)]
    })
    return x_user

In [18]:
get_user_df(13.4, "male", "no", "thur", "dinner", 4)

,total_bill,sex,smoker,day,time,size
0,13.4,Male,No,Thur,Dinner,4


In [19]:
pickled_model.predict(get_user_df(13.4, "male", "no", "thur", "dinner", 4))

array([2.609375])